**Import dependencies**

In [63]:
import pandas as pd
import numpy as np
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score

In [64]:
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier

**Load dataset**

In [65]:
df= pd.read_csv('https://github.com/V-Vibee/Pro_Smart/raw/main/2.0%20Right%20Model/heart.csv')
df.head()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,63,1,3,145,233,1,0,150,0,2.3,0,0,1,1
1,37,1,2,130,250,0,1,187,0,3.5,0,0,2,1
2,41,0,1,130,204,0,0,172,0,1.4,2,0,2,1
3,56,1,1,120,236,0,1,178,0,0.8,2,0,2,1
4,57,0,0,120,354,0,1,163,1,0.6,2,0,2,1


In [66]:
df.shape

(303, 14)

In [67]:
df.isnull().sum()

,0
age,0
sex,0
cp,0
trestbps,0
chol,0
fbs,0
restecg,0
thalach,0
exang,0
oldpeak,0


In [68]:
df.columns

Index(['age', 'sex', 'cp', 'trestbps', 'chol', 'fbs', 'restecg', 'thalach',
       'exang', 'oldpeak', 'slope', 'ca', 'thal', 'target'],
      dtype='object')

In [69]:
df['target'].value_counts()

,count
target,
1,165
0,138


1-----> Defective Heart

0-----> Healthy Heart

**Split Features & Target**

In [70]:
X= df.drop(columns='target', axis=1)
y= df['target']

In [71]:
X= np.asarray(X)
y= np.asarray(y)

**Model Selection**

1. Comparing model with Default Hyperparameters using cross validation

---



In [72]:
models= [LogisticRegression(max_iter=1000), SVC(kernel='linear'),KNeighborsClassifier(),RandomForestClassifier(random_state=0)]

In [73]:
def compare_models_cross_validation():

  for model in models:

    cv_score= cross_val_score(model,X,y, cv=5)
    mean_accuracy= sum(cv_score)/ len(cv_score)
    mean_accuracy = mean_accuracy*100
    mean_accuracy= round(mean_accuracy,2)

    print('Cross validation accuracy of ', model, '=', cv_score)
    print('Accuracy score of the model ', model, '=', mean_accuracy, '%')
    print('----------------------------------------------------------------------------------------')

In [74]:
compare_models_cross_validation()

Cross validation accuracy of  LogisticRegression(max_iter=1000) = [0.80327869 0.86885246 0.85245902 0.86666667 0.75      ]
Accuracy score of the model  LogisticRegression(max_iter=1000) = 82.83 %
----------------------------------------------------------------------------------------
Cross validation accuracy of  SVC(kernel='linear') = [0.81967213 0.8852459  0.80327869 0.86666667 0.76666667]
Accuracy score of the model  SVC(kernel='linear') = 82.83 %
----------------------------------------------------------------------------------------
Cross validation accuracy of  KNeighborsClassifier() = [0.60655738 0.6557377  0.57377049 0.73333333 0.65      ]
Accuracy score of the model  KNeighborsClassifier() = 64.39 %
----------------------------------------------------------------------------------------
Cross validation accuracy of  RandomForestClassifier(random_state=0) = [0.85245902 0.90163934 0.81967213 0.81666667 0.8       ]
Accuracy score of the model  RandomForestClassifier(random_state=

**Random Forest has got the best Accuracy score**

2. Comparing the models with different Hyperparameter values using GridSearch CV

In [75]:
# list of models

model_list= [LogisticRegression(max_iter=10000), SVC(),KNeighborsClassifier(),RandomForestClassifier(random_state=0)]


In [76]:
# Creating dictionary, that contaings Hyperparameter values


model_hyperparameters= {
    'log_reg_hyperparameters':{
        'C':[1,5,10,20]
    },

    'svc_hyperparameters': {
        'kernel':['linear','poly','rbf','sigmoid'],
        'C': [1,5,10,20]
    },

    'knn_hyperparameters':{
        'n_neighbors':[3,5,10]
    },

    'rfc_hyperparameters':{
        'n_estimators':[10,20,50,100]
          }
}

In [77]:
a= {'Name':['John','Sam'], 'Age':30, 'Country':'India'}
print(a)

{'Name': ['John', 'Sam'], 'Age': 30, 'Country': 'India'}


In [78]:
type(model_hyperparameters)

dict

In [79]:
print(model_hyperparameters.keys())

dict_keys(['log_reg_hyperparameters', 'svc_hyperparameters', 'knn_hyperparameters', 'rfc_hyperparameters'])


In [80]:
model_keys= list(model_hyperparameters.keys())
print(model_keys)

['log_reg_hyperparameters', 'svc_hyperparameters', 'knn_hyperparameters', 'rfc_hyperparameters']


In [81]:
model_hyperparameters['svc_hyperparameters']

{'kernel': ['linear', 'poly', 'rbf', 'sigmoid'], 'C': [1, 5, 10, 20]}

In [82]:
model_keys[0]

'log_reg_hyperparameters'

In [83]:
model_keys[1]

'svc_hyperparameters'

In [84]:
model_hyperparameters[model_keys[1]]

{'kernel': ['linear', 'poly', 'rbf', 'sigmoid'], 'C': [1, 5, 10, 20]}

**Gridsearch CV**

In [96]:
def model_selection(list_of_models, hyperparameters_dictionary):
    result = []

    for i, model in enumerate(list_of_models):
        key = model_keys[i]
        params = hyperparameters_dictionary[key]

        print(f"Model: {model}")
        print(f"Hyperparameters: {params}")
        print('-------------------------------')

        classifier = GridSearchCV(model, params, cv=5)

        # fitting data to classifier
        classifier.fit(X, y)

        result.append({
            'model used': model,
            'highest score': classifier.best_score_,
            'best hyperparameters': classifier.best_params_
        })

    result_dataframe = pd.DataFrame(result, columns=['model used', 'highest score', 'best hyperparameters'])

    # Print or return the dataframe
    print(result_dataframe)
    return result_dataframe

In [97]:
model_selection(model_list,model_hyperparameters)

Model: LogisticRegression(max_iter=10000)
Hyperparameters: {'C': [1, 5, 10, 20]}
-------------------------------
Model: SVC()
Hyperparameters: {'kernel': ['linear', 'poly', 'rbf', 'sigmoid'], 'C': [1, 5, 10, 20]}
-------------------------------
Model: KNeighborsClassifier()
Hyperparameters: {'n_neighbors': [3, 5, 10]}
-------------------------------
Model: RandomForestClassifier(random_state=0)
Hyperparameters: {'n_estimators': [10, 20, 50, 100]}
-------------------------------
                               model used  highest score  \
0      LogisticRegression(max_iter=10000)       0.831585   
1                                   SVC()       0.828306   
2                  KNeighborsClassifier()       0.643880   
3  RandomForestClassifier(random_state=0)       0.838087   

           best hyperparameters  
0                      {'C': 5}  
1  {'C': 1, 'kernel': 'linear'}  
2            {'n_neighbors': 5}  
3         {'n_estimators': 100}  


,model used,highest score,best hyperparameters
0,LogisticRegression(max_iter=10000),0.831585,{'C': 5}
1,SVC(),0.828306,"{'C': 1, 'kernel': 'linear'}"
2,KNeighborsClassifier(),0.643880,{'n_neighbors': 5}
3,RandomForestClassifier(random_state=0),0.838087,{'n_estimators': 100}
